In [5]:
import plotly.graph_objs as go
import pandas as pd
import folium
import numpy as np
import plotly.offline as off

off.init_notebook_mode(connected=True)

df = pd.read_csv('fastfoodmaps_locations_2007.csv')
df.loc['Name']= df['Type']
df.loc[df['Type']== 'b','Name']='Burger King'
df.loc[df['Type']== 'm','Name']='McDonalds'
df.loc[df['Type']== 'p','Name']='Pizza Hut'
df.loc[df['Type']== 't','Name']='Taco Bell'
df.loc[df['Type']== 'w','Name']="Wendy's"
df.loc[df['Type']== 'k','Name']='KFC'
df.loc[df['Type']== 'j','Name']='Jack'
df.loc[df['Type']== 'c','Name']="Carl's Jr."
df.loc[df['Type']== 'i','Name']='In-N-Out'
df.loc[df['Type']== 'h','Name']="Hardee's"
df['text'] = df['Name']+ '<br>'+ df['Location'] + ', ' + df['State'] + ', ' + df['Zip']
# df.head(10)

In [4]:
def addWindow(df, storeType1, storeType2):   
    df_B = df.loc[df['Type']==storeType1]
    df_BK = df_B[['Name','Type','latitude','longitude','State']].copy()
    df_BK.sort_values(by='latitude', ascending = False, inplace=True)

    # generate window's boundary for 1 mile
    coef = 0.0145
    df_BK['latitude_lower'] = df_BK['latitude']-coef
    df_BK['latitude_upper'] = df_BK['latitude']+coef
    df_BK['longitude_lower'] = df_BK['longitude']-coef
    df_BK['longitude_upper'] = df_BK['longitude']+coef

    df_m = df.loc[df['Type']==storeType2]
    df_M = df_m[['Name','Type','latitude','longitude','State']].copy()
    df_M.sort_values(by='latitude', ascending = False, inplace = True)
    
    df_new = pd.concat([df_BK, df_M], axis=0, ignore_index=True, sort = False)
    
    return df_new



def setupFlag(df_new, storeType):
    df_new['flag']=0
    cutoff = df_new[df_new['Type']==storeType].shape[0]
    start = 0

    for k in range(len(df_new[df_new['Type']==storeType])):
        for j in range(start, len(df_new[df_new['Type']!=storeType])):
            if  df_new['latitude'][cutoff+j] >=df_new['latitude_lower'][k] \
            and df_new['latitude'][cutoff+j] <=df_new['latitude_upper'][k]\
            and df_new['longitude'][cutoff+j]>=df_new['longitude_lower'][k] \
            and df_new['longitude'][cutoff+j]<=df_new['longitude_upper'][k] :
                df_new['flag'][k]=1
                df_new['flag'][cutoff+j]=1
                start = j
                break
    return df_new 
    

def makeColorlist(df):
    conditions = [
         (df['Type'] == 'm'),
         (df['Type'] == 'b'),
         (df['Type'] == 'w'), 
         (df['Type'] == 'k')
    ]

    colorlist = ["blue", "green", "red","gray"]
    df["color"] = np.select(conditions, colorlist)
    return df


def gacompare (dfs,mapNew):
    for row in dfs.itertuples():
        mapNew.add_child(folium.Marker(location = [row.latitude,row.longitude],
                                      popup = row.Name,
                                      icon = folium.Icon(color=row.color,
                                                          prefix='fa',icon="circle")
                                 ))

    legend_html =   '''
                    <div style="position: fixed; 
                                bottom: 50px; left: 50px; width: 120px; height: 150px; 
                                border:3px solid grey; background-color:rgba(255, 255, 255, 0.8);
                                z-index:9999; font-size:14px;
                                ">&nbsp; Fastfood Name <br>
                                <i class="fa fa-map-marker fa-2x" style="color:green"></i> &nbsp; BurgerKing &nbsp; <br>
                                <i class="fa fa-map-marker fa-2x" style="color:blue"></i>  &nbsp; McDonalds  &nbsp; <br>
                                <i class="fa fa-map-marker fa-2x" style="color:red"></i>   &nbsp; Wendy's &nbsp; <br>
                                <i class="fa fa-map-marker fa-2x" style="color:gray"></i>      &nbsp; KFC &nbsp;
                    </div>
                    ''' 

    mapNew.get_root().html.add_child(folium.Element(legend_html))
    return mapNew

Pair 4 stores 

In [6]:
# #### pair 'Burger King' and 'McDonald'  #### #

df_M_BK = addWindow(df, 'b', 'm')
df_M_BK = setupFlag(df_M_BK, 'b')
df_M_BK = makeColorlist(df_M_BK)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [11]:
# #### pair 'Wendy's' and 'McDonald' #### #

df_M_W = addWindow(df, 'w', 'm')
df_M_W = setupFlag(df_M_W, 'w')
df_M_W = makeColorlist(df_M_W)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [15]:
# #### pair 'KFC' and 'McDonald' #### #

df_M_K = addWindow(df, 'k', 'm')
df_M_K = setupFlag(df_M_K, 'k')
df_M_K = makeColorlist(df_M_K)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



# Display store pairs in GA

In [7]:
# 'Burger King' and 'McDonald'
mapGA = folium.Map(location=[32.753746, -83.386330],zoom_start=7)

dfgaMB = df_M_BK[df_M_BK["State"]=="GA"][df_M_BK["flag"]==1]
map_MB=gacompare(dfgaMB,mapGA)
map_MB

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [12]:
# 'Wendy's' and 'McDonald'
mapGA = folium.Map(location=[32.753746, -83.386330],zoom_start=7)

dfgaWM = df_M_W[df_M_W["State"]=="GA"][df_M_W["flag"]==1]
map_WM=gacompare(dfgaWM,mapGA)
map_WM

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [16]:
# 'KFC' and 'McDonald'
mapGA = folium.Map(location=[32.753746, -83.386330],zoom_start=7)

dfgaKM = df_M_K[df_M_K["State"]=="GA"][df_M_K["flag"]==1]
map_KM=gacompare(dfgaKM,mapGA)
map_KM

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



# Display store pairs in AK

In [17]:
# 'Burger King' and 'McDonald'
mapAK = folium.Map(location=[61.1811, -149.868],zoom_start=5)

dfgaMB_AK = df_M_BK[df_M_BK["State"]=="AK"][df_M_BK["flag"]==1]
map_MB_AK=gacompare(dfgaMB_AK,mapAK)
map_MB_AK

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [13]:
# 'Wendy's' and 'McDonald'
mapAK = folium.Map(location=[61.1811, -149.868],zoom_start=5)

dfgaMW_AK = df_M_W[df_M_W["State"]=="AK"][df_M_W["flag"]==1]
map_MW_AK=gacompare(dfgaMW_AK,mapAK)
map_MW_AK      

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [18]:
# 'KFC' and 'McDonald'
mapAK = folium.Map(location=[61.1811, -149.868],zoom_start=5)

dfgaMK_AK = df_M_K[df_M_K["State"]=="AK"][df_M_K["flag"]==1]
map_MK_AK=gacompare(dfgaMK_AK,mapAK)
map_MK_AK

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



# Display store pairs in CA

In [10]:
# 'Burger King' and 'McDonald'
mapCA = folium.Map(location=[37, -120],zoom_start=6)

dfgaMB_CA = df_M_BK[df_M_BK["State"]=="CA"][df_M_BK["flag"]==1]
map_MB_CA=gacompare(dfgaMB_CA,mapCA)
map_MB_CA

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [14]:
# 'Wendy's' and 'McDonald'
mapCA = folium.Map(location=[37, -120],zoom_start=6)

dfgaMW_CA = df_M_W[df_M_W["State"]=="CA"][df_M_W["flag"]==1]
map_MW_CA=gacompare(dfgaMW_CA,mapCA)
map_MW_CA  

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [44]:
# 'KFC' and 'McDonald'
mapCA = folium.Map(location=[37, -120],zoom_start=6)

dfgaMK_CA = df_M_K[df_M_K["State"]=="CA"][df_M_K["flag"]==1]
map_MK_CA=gacompare(dfgaMK_CA,mapCA)
map_MK_CA

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [222]:
storelist=["McDonalds","Burger King","Wendy's","KFC"]
statelist = ['AK','GA','CA']
dfa=df[df["Name"].isin(storelist)]
USANumber=dfa.groupby(["Name"],as_index=False).count()
USANumber= pd.DataFrame({'Name':USANumber['Name'],'State':'USA','count':USANumber['Number'] })

stateNumber = dfa.groupby(['Name', 'State']).size().reset_index(name='count')
stateNumber=stateNumber[stateNumber['State'].isin(statelist)]


OverallNumber = pd.concat([stateNumber, USANumber], axis=0, ignore_index=True, sort = False)
OverallNumber=OverallNumber.sort_values('Name')

In [228]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
# df_M_BK_1=df_M_BK.drop(['latitude','longitude','latitude_lower','latitude_upper',
#               'longitude_lower','longitude_upper','color'],axis=1)
# df_M_BK_flag = df_M_BK_1[df_M_BK_1['flag']==1][df_M_BK_1['Type']!='m']
# df_M_BK_flag

df_M_BK_flag = df_M_BK[df_M_BK['flag']==1][df_M_BK['Type']!='m']
df_M_BK_flag

df_M_W_1=df_M_W.drop(['latitude','longitude','latitude_lower','latitude_upper',
               'longitude_lower','longitude_upper','color'],axis=1)
df_M_W_flag = df_M_W_1[df_M_W_1['flag']==1][df_M_W_1['Type']!='m']
df_M_W_flag

df_M_K_1=df_M_K.drop(['latitude','longitude','latitude_lower','latitude_upper',
               'longitude_lower','longitude_upper','color'],axis=1)
df_M_K_flag = df_M_K_1[df_M_K_1['flag']==1][df_M_K_1['Type']!='m']
df_M_K_flag


statelist = ['AK','GA','CA']
USA_BK=df_M_BK_flag.groupby(["Name"],as_index=False).count()
USA_BK= pd.DataFrame({'Name':USA_BK['Name'],'State':'USA','count':USA_BK['flag'] })
BK = df_M_BK_flag.groupby(['Name','State']).size().reset_index(name='count')
BK =BK [BK['State'].isin(statelist)]

USA_WD= df_M_W_flag.groupby(["Name"],as_index=False).count()
USA_WD= pd.DataFrame({'Name':USA_WD['Name'],'State':'USA','count':USA_WD['flag'] })
WD = df_M_W_flag.groupby(['Name','State']).size().reset_index(name='count')
WD =WD [WD['State'].isin(statelist)]

USA_KFC= df_M_K_flag.groupby(["Name"],as_index=False).count()
USA_KFC= pd.DataFrame({'Name':USA_KFC['Name'],'State':'USA','count':USA_KFC['flag'] })
KFC = df_M_K_flag.groupby(['Name','State']).size().reset_index(name='count')
KFC =KFC [KFC['State'].isin(statelist)]


In [229]:
NumTab = pd.concat([USA_BK, BK,USA_WD,WD,USA_KFC,KFC], axis=0, ignore_index=True, sort = True)
NumTab = NumTab.sort_values(['Name','State'])
totalTab = OverallNumber[OverallNumber['Name']!='McDonalds']
totalTab['pair']=NumTab['count'].tolist()
totalTab['percent_%']=totalTab['pair']/totalTab['count']*100

In [199]:
from plotly import tools

lmt = ['AK','GA','CA']#,'USA']
x=["Burger King","Wendy's","KFC"]

fig1 = go.Figure()
for k in lmt:
    y = totalTab['count'][totalTab['State']==k]
    fig1.add_trace(go.Bar(x=x, y=y,name = k))

fig1.layout.update(title_text='Total Store in the States',
    xaxis=dict(title="Stores"),
    yaxis=dict(title="Number of Store"))
fig1.show()

In [223]:
fig2 = go.Figure()
for k in lmt:
    y = totalTab['pair'][totalTab['State']==k]
    fig2.add_trace(go.Bar(x=x, y=y,name = k))
fig2.layout.update(title_text='Number of Store Paired with McDonalds in the States',
    xaxis=dict(title="Stores"),
    yaxis=dict(title="Number of Paired Store"))
fig2.show()


In [224]:
lmt1 = ['AK','GA','CA','USA']
fig3 = go.Figure()
for k in lmt1:
    y = totalTab['percent_%'][totalTab['State']==k]
    fig3.add_trace(go.Scatter(x=x, y=y,name = k, mode = 'markers + lines'))
    
fig3.layout.update(title_text='Percentage of Stores Paired with McDonlads ',
    xaxis=dict(title="Stores"),
    yaxis=dict(title="Paired Percentage"))
fig3.show()

In [2]:
stateNumber = df.groupby(['Name', 'State']).size().reset_index(name='count')
x = stateNumber['State']

lmt=["McDonalds","Burger King","Wendy's","KFC"]

fig = go.Figure()
for k in lmt:
    y = stateNumber['count'][stateNumber['Name']==k]
    fig.add_trace(go.Bar(x=x, y=y,name = k))

fig.layout.update(barmode='relative', title_text='Numbers of the Stores in Each State',
    xaxis=dict(title="State"),
    yaxis=dict(title="Fastfood Brand"))
fig.show()